# BERT를 사용한 종단 간 마스크 언어 모델링

**Author:** [Ankur Singh](https://twitter.com/ankur310794)<br>
**Date created:** 2020/09/18<br>
**Last modified:** 2020/09/18<br>
**Description:** BERT를 사용하여 MLM(Masked Language Model)을 구현하고 IMDB 리뷰 데이터 세트에서 미세 조정합니다.

## 소개

Masked Language Modeling은 빈칸 채우기 작업으로, 모델이 마스크 토큰을 둘러싼 컨텍스트 단어를 사용하여 마스크된 단어가 무엇인지 예측하려고 시도합니다.

하나 이상의 마스크 토큰이 포함된 입력의 경우 모델은 각각에 대해 가장 가능성이 높은 대체를 생성합니다.

Example:

- Input: "I have watched this [MASK] and it was awesome."
- Output: "I have watched this movie and it was awesome."

마스크된 언어 모델링은 셀프 지도학습 설정(사람이 주석 처리한 레이블 없음)에서 언어 모델을 훈련하는 좋은 방법입니다. 그런 다음 이러한 모델을 미세 조정하여 다양한 지도학습 NLP 작업을 수행할 수 있습니다.

이 예제는 BERT 모델을 처음부터 구축하고, 마스크된 언어 모델링 작업으로 훈련시킨 다음, 감정 분류 작업에서 이 모델을 미세 조정하는 방법을 알려줍니다.

Keras `TextVectorization`와 `MultiHeadAttention`레이어를 사용하여 BERT Transformer-Encoder 네트워크 아키텍처를 생성합니다.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from dataclasses import dataclass
import pandas as pd
import numpy as np
import glob
import re
from pprint import pprint

## 설정

In [2]:
@dataclass
class Config:
    MAX_LEN = 256
    BATCH_SIZE = 32
    LR = 0.001
    VOCAB_SIZE = 30000
    EMBED_DIM = 128
    NUM_HEAD = 8  # used in bert model
    FF_DIM = 128  # used in bert model
    NUM_LAYERS = 1

config = Config()

## 데이터 로드

먼저 IMDB 데이터를 다운로드하고 Pandas 데이터 프레임에 로드합니다.

In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 80.2M    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 80.2M    0  160k    0     0   105k      0  0:12:58  0:00:01  0:12:57  105k
  0 80.2M    0  624k    0     0   251k      0  0:05:26  0:00:02  0:05:24  251k
  1 80.2M    1 1584k    0     0   450k      0  0:03:02  0:00:03  0:02:59  450k
  3 80.2M    3 3136k    0     0   701k      0  0:01:57  0:00:04  0:01:53  701k
  6 80.2M    6 5744k    0     0  1050k      0  0:01:18  0:00:05  0:01:13 1177k
 11 80.2M   11 9056k    0     0  1400k      0  0:00:58  0:00:06  0:00:52 1796k
 16 80.2M   16 13.0M    0     0  1788k      0  0:00:45  0:00:07  0:00:38 2551k
 23 80.2M   23 18.9M    0     0  2288k      0  0:00:35  0:00:08  0:00:27 3595k
 33 80.2M   33 27.0M    0     0  2925k      0  0:00

In [4]:
def get_text_list_from_files(files):
    text_list = []
    for name in files:
        with open(name, encoding='UTF-8') as f:
            for line in f:
                text_list.append(line)
    return text_list


def get_data_from_text_files(folder_name):

    pos_files = glob.glob("aclImdb/" + folder_name + "/pos/*.txt")
    pos_texts = get_text_list_from_files(pos_files)
    neg_files = glob.glob("aclImdb/" + folder_name + "/neg/*.txt")
    neg_texts = get_text_list_from_files(neg_files)
    df = pd.DataFrame(
        {
            "review": pos_texts + neg_texts,
            "sentiment": [0] * len(pos_texts) + [1] * len(neg_texts),
        }
    )
    df = df.sample(len(df)).reset_index(drop=True)
    return df


train_df = get_data_from_text_files("train")
test_df = get_data_from_text_files("test")

all_data = train_df.append(test_df)

## 데이터 세트 준비

`TextVectorization` 레이어를 사용 하여 텍스트를 정수 토큰 ID로 벡터화합니다. 문자열 배치를 토큰 인덱스 시퀀스(순서대로 하나의 샘플 = 정수 토큰 인덱스의 1D 배열) 또는 조밀한 표현(하나의 샘플 = 정렬되지 않은 토큰 세트를 인코딩하는 부동 소수점 값의 1D 배열)으로 변환합니다.

아래에서는 3개의 전처리 기능을 정의합니다.

1.  `get_vectorize_layer`함수는 `TextVectorization`레이어를 만듭니다.
2.  `encode` 함수는 원시 텍스트를 정수 토큰 ID로 인코딩합니다.
3.  `get_masked_input_and_labels`함수는 입력 토큰 ID를 마스킹합니다. 무작위로 각 시퀀스의 모든 입력 토큰의 15%를 마스킹합니다.

In [24]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape("!#$%&'()*+,-./:;<=>?@\^_`{|}~"), ""
    )


def get_vectorize_layer(texts, vocab_size, max_seq, special_tokens=["[MASK]"]):
    """텍스트 벡터화 레이어 구축

    Args:
      texts (list): 문자열 목록, 즉 입력 텍스트
      vocab_size (int): 어휘 크기
      max_seq (int): 최대 시퀀스 길이.
      special_tokens (list, optional): 특수 토큰 목록입니다. 기본값은 ['[MASK]']입니다.

    Returns:
        layers.Layer: TextVectorization Keras 레이어 반환
    """
    vectorize_layer = TextVectorization(
        max_tokens=vocab_size,
        output_mode="int",
        standardize=custom_standardization,
        output_sequence_length=max_seq,
    )
    vectorize_layer.adapt(texts)

    # 어휘에 마스크 토큰 삽입
    vocab = vectorize_layer.get_vocabulary()
    vocab = vocab[2 : vocab_size - len(special_tokens)] + ["[mask]"]
    vectorize_layer.set_vocabulary(vocab)
    return vectorize_layer


vectorize_layer = get_vectorize_layer(
    all_data.review.values.tolist(),
    config.VOCAB_SIZE,
    config.MAX_LEN,
    special_tokens=["[mask]"],
)

# 마스크된 언어 모델에 대한 마스크 토큰 ID 가져오기
mask_token_id = vectorize_layer(["[mask]"]).numpy()[0][0]

def encode(texts):
    encoded_texts = vectorize_layer(texts)
    return encoded_texts.numpy()


def get_masked_input_and_labels(encoded_texts):
    # 15% BERT 마스킹
    inp_mask = np.random.rand(*encoded_texts.shape) < 0.15
    # 특수 토큰을 마스킹하지 마십시오.
    inp_mask[encoded_texts <= 2] = False
    # 기본적으로 대상을 -1로 설정합니다. 무시를 의미합니다.
    labels = -1 * np.ones(encoded_texts.shape, dtype=int)
    # 마스킹된 토큰에 대한 레이블 설정
    labels[inp_mask] = encoded_texts[inp_mask]

    # 입력 준비
    encoded_texts_masked = np.copy(encoded_texts)
    # 90%의 토큰에 대한 마지막 토큰인 [MASK]에 입력을 설정합니다.
    # 이것은 10%를 그대로 두는 것을 의미합니다.
    inp_mask_2mask = inp_mask & (np.random.rand(*encoded_texts.shape) < 0.90)
    encoded_texts_masked[
        inp_mask_2mask
    ] = mask_token_id  # 마스크 토큰은 dict의 마지막입니다.

    # 10%를 임의의 토큰으로 설정
    inp_mask_2random = inp_mask_2mask & (np.random.rand(*encoded_texts.shape) < 1 / 9)
    encoded_texts_masked[inp_mask_2random] = np.random.randint(
        3, mask_token_id, inp_mask_2random.sum()
    )

    # .fit() 메서드에 전달할 sample_weights 준비
    sample_weights = np.ones(labels.shape)
    sample_weights[labels == -1] = 0

    # y_labels는 encode_texts, 즉 입력 토큰과 동일합니다.
    y_labels = np.copy(encoded_texts)

    return encoded_texts_masked, y_labels, sample_weights


# 훈련을 위한 25000개의 예제가 있습니다.
x_train = encode(train_df.review.values)  # 벡터라이저로 리뷰 인코딩
y_train = train_df.sentiment.values
train_classifier_ds = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(1000)
    .batch(config.BATCH_SIZE)
)

# 테스트를 위한 25000개의 예제가 있습니다.
x_test = encode(test_df.review.values)
y_test = test_df.sentiment.values
test_classifier_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(
    config.BATCH_SIZE
)

# 종단 간 모델 입력을 위한 데이터 세트 구축(마지막에 사용됨)
test_raw_classifier_ds = tf.data.Dataset.from_tensor_slices(
    (test_df.review.values, y_test)
).batch(config.BATCH_SIZE)

# 마스킹된 언어 모델에 대한 데이터 준비
x_all_review = encode(all_data.review.values)
x_masked_train, y_masked_labels, sample_weights = get_masked_input_and_labels(
    x_all_review
)

mlm_ds = tf.data.Dataset.from_tensor_slices(
    (x_masked_train, y_masked_labels, sample_weights)
)
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)

## 마스크 언어 모델링을 위한 BERT 모델(Pretraining Model) 생성

레이어 를 사용하여 BERT와 같은 사전 학습 모델 아키텍처를 생성합니다 `MultiHeadAttention`. 토큰 ID를 입력(마스킹된 토큰 포함)으로 사용하고 마스크된 입력 토큰의 올바른 ID를 예측합니다.

In [25]:

def bert_module(query, key, value, i):
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=config.NUM_HEAD,
        key_dim=config.EMBED_DIM // config.NUM_HEAD,
        name="encoder_{}/multiheadattention".format(i),
    )(query, key, value)
    attention_output = layers.Dropout(0.1, name="encoder_{}/att_dropout".format(i))(
        attention_output
    )
    attention_output = layers.LayerNormalization(
        epsilon=1e-6, name="encoder_{}/att_layernormalization".format(i)
    )(query + attention_output)

    # Feed-forward layer
    ffn = keras.Sequential(
        [
            layers.Dense(config.FF_DIM, activation="relu"),
            layers.Dense(config.EMBED_DIM),
        ],
        name="encoder_{}/ffn".format(i),
    )
    ffn_output = ffn(attention_output)
    ffn_output = layers.Dropout(0.1, name="encoder_{}/ffn_dropout".format(i))(
        ffn_output
    )
    sequence_output = layers.LayerNormalization(
        epsilon=1e-6, name="encoder_{}/ffn_layernormalization".format(i)
    )(attention_output + ffn_output)
    return sequence_output


def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc


loss_fn = keras.losses.SparseCategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE
)
loss_tracker = tf.keras.metrics.Mean(name="loss")


class MaskedLanguageModel(tf.keras.Model):
    def train_step(self, inputs):
        if len(inputs) == 3:
            features, labels, sample_weight = inputs
        else:
            features, labels = inputs
            sample_weight = None

        with tf.GradientTape() as tape:
            predictions = self(features, training=True)
            loss = loss_fn(labels, predictions, sample_weight=sample_weight)

        # 그라디언트 계산
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # 가중치 업데이트
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # 메트릭 계산
        loss_tracker.update_state(loss, sample_weight=sample_weight)

        # 메트릭 이름을 현재 값으로 매핑하는 dict 반환
        return {"loss": loss_tracker.result()}

    @property
    def metrics(self):
        # `reset_states()`가 될 수 있도록 `Metric` 객체를 여기에 나열합니다.
        # 각 Epoch 시작 시 자동으로 호출됨
        # 또는 `evaluate()`의 시작 부분에서.
        # 이 속성을 구현하지 않으면 다음을 호출해야 합니다.
        # 선택한 시간에 `reset_states()`
        return [loss_tracker]


def create_masked_language_bert_model():
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)

    word_embeddings = layers.Embedding(
        config.VOCAB_SIZE, config.EMBED_DIM, name="word_embedding"
    )(inputs)
    position_embeddings = layers.Embedding(
        input_dim=config.MAX_LEN,
        output_dim=config.EMBED_DIM,
        weights=[get_pos_encoding_matrix(config.MAX_LEN, config.EMBED_DIM)],
        name="position_embedding",
    )(tf.range(start=0, limit=config.MAX_LEN, delta=1))
    embeddings = word_embeddings + position_embeddings

    encoder_output = embeddings
    for i in range(config.NUM_LAYERS):
        encoder_output = bert_module(encoder_output, encoder_output, encoder_output, i)

    mlm_output = layers.Dense(config.VOCAB_SIZE, name="mlm_cls", activation="softmax")(
        encoder_output
    )
    mlm_model = MaskedLanguageModel(inputs, mlm_output, name="masked_bert_model")

    optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    mlm_model.compile(optimizer=optimizer)
    return mlm_model


id2token = dict(enumerate(vectorize_layer.get_vocabulary()))
token2id = {y: x for x, y in id2token.items()}


class MaskedTextGenerator(keras.callbacks.Callback):
    def __init__(self, sample_tokens, top_k=5):
        self.sample_tokens = sample_tokens
        self.k = top_k

    def decode(self, tokens):
        return " ".join([id2token[t] for t in tokens if t != 0])

    def convert_ids_to_tokens(self, id):
        return id2token[id]

    def on_epoch_end(self, epoch, logs=None):
        prediction = self.model.predict(self.sample_tokens)

        masked_index = np.where(self.sample_tokens == mask_token_id)
        masked_index = masked_index[1]
        mask_prediction = prediction[0][masked_index]

        top_indices = mask_prediction[0].argsort()[-self.k :][::-1]
        values = mask_prediction[0][top_indices]

        for i in range(len(top_indices)):
            p = top_indices[i]
            v = values[i]
            tokens = np.copy(sample_tokens[0])
            tokens[masked_index[0]] = p
            result = {
                "input_text": self.decode(sample_tokens[0].numpy()),
                "prediction": self.decode(tokens),
                "probability": v,
                "predicted mask token": self.convert_ids_to_tokens(p),
            }
            pprint(result)


sample_tokens = vectorize_layer(["I have watched this [mask] and it was awesome"])
generator_callback = MaskedTextGenerator(sample_tokens.numpy())

bert_masked_model = create_masked_language_bert_model()
bert_masked_model.summary()

Model: "masked_bert_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 word_embedding (Embedding)     (None, 256, 128)     3840000     ['input_1[0][0]']                
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 256, 128)    0           ['word_embedding[0][0]']         
 da)                                                                                              
                                                                                                  
 encoder_0/multiheadattention (  (None, 256, 128)    66048       ['tf.__operators_

## 훈련과 저장

In [26]:
bert_masked_model.fit(mlm_ds, epochs=5, callbacks=[generator_callback])
bert_masked_model.save("bert_mlm_imdb.h5")

Epoch 1/5
1563/1563 [==============================] - ETA: 0s - loss: 6.9901{'input_text': 'i have watched this [mask] and it was awesome',
 'predicted mask token': 'this',
 'prediction': 'i have watched this this and it was awesome',
 'probability': 0.048747342}
{'input_text': 'i have watched this [mask] and it was awesome',
 'predicted mask token': 'a',
 'prediction': 'i have watched this a and it was awesome',
 'probability': 0.043583322}
{'input_text': 'i have watched this [mask] and it was awesome',
 'predicted mask token': 'i',
 'prediction': 'i have watched this i and it was awesome',
 'probability': 0.03734729}
{'input_text': 'i have watched this [mask] and it was awesome',
 'predicted mask token': 'to',
 'prediction': 'i have watched this to and it was awesome',
 'probability': 0.032652}
{'input_text': 'i have watched this [mask] and it was awesome',
 'predicted mask token': 'of',
 'prediction': 'i have watched this of and it was awesome',
 'probability': 0.029586175}
1563/15

## 감정 분류 모델 미세 조정

감정 분류의 다운스트림 작업에서 자체 지도 모델을 미세 조정할 것입니다. 이를 위해 `Dense`사전 훈련된 BERT 기능 위에 풀링 계층과 계층을 추가하여 분류기를 생성해 보겠습니다.

In [27]:
# 사전 훈련된 bert 모델 불러오기
mlm_model = keras.models.load_model(
    "bert_mlm_imdb.h5", custom_objects={"MaskedLanguageModel": MaskedLanguageModel}
)
pretrained_bert_model = tf.keras.Model(
    mlm_model.input, mlm_model.get_layer("encoder_0/ffn_layernormalization").output
)

# 동결
pretrained_bert_model.trainable = False


def create_classifier_bert_model():
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)
    sequence_output = pretrained_bert_model(inputs)
    pooled_output = layers.GlobalMaxPooling1D()(sequence_output)
    hidden_layer = layers.Dense(64, activation="relu")(pooled_output)
    outputs = layers.Dense(1, activation="sigmoid")(hidden_layer)
    classifer_model = keras.Model(inputs, outputs, name="classification")
    optimizer = keras.optimizers.Adam()
    classifer_model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return classifer_model


classifer_model = create_classifier_bert_model()
classifer_model.summary()

# 고정된 BERT 단계로 분류기 훈련
classifer_model.fit(
    train_classifier_ds,
    epochs=5,
    validation_data=test_classifier_ds,
)

# 미세 조정을 위해 BERT 모델 고정 해제
pretrained_bert_model.trainable = True
optimizer = keras.optimizers.Adam()
classifer_model.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
)
classifer_model.fit(
    train_classifier_ds,
    epochs=5,
    validation_data=test_classifier_ds,
)

Model: "classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256)]             0         
                                                                 
 model (Functional)          (None, 256, 128)          3939584   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,947,905
Trainable params: 8,321
Non-trainable params: 3,939,584
______________________________________

## 종단 간 모델 생성 및 평가

모델을 배포하려는 경우 프로덕션 환경에서 사전 처리 논리를 다시 구현할 필요가 없도록 사전 처리 파이프라인이 이미 포함되어 있는 것이 가장 좋습니다. `TextVectorization`레이어 를 통합하는 종단 간 모델을 만들고 평가해 보겠습니다. 우리 모델은 원시 문자열을 입력으로 받아들입니다.

In [28]:

def get_end_to_end(model):
    inputs_string = keras.Input(shape=(1,), dtype="string")
    indices = vectorize_layer(inputs_string)
    outputs = model(indices)
    end_to_end_model = keras.Model(inputs_string, outputs, name="end_to_end_model")
    optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    end_to_end_model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return end_to_end_model


end_to_end_classification_model = get_end_to_end(classifer_model)
end_to_end_classification_model.evaluate(test_raw_classifier_ds)

782/782 [==============================] - 6s 7ms/step - loss: 0.7381 - accuracy: 0.8226


[0.7381284236907959, 0.8225600123405457]